In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

# Task description

To train a variable misuse detection model one needs to implement an NLP labeling model.

For example, for a function containing misuse
```
def _eq(l1, l2):\n    return (set(l1) == set(l1))
```
the misuse character span is (44, 46). To do this with NLP methods, code is tokenized, and labels for tokens are generated
```
[def, _, eq, (, l, 1, ",", l, 2, ):, \n, \t, return, (, set, (, l1, ), ==, set, (, l1, ), ), ]
[O  , O, O , O, O, O,  O , O, O, 0 , O , O ,    O  , O, O  , O, O , O, O , O  , O, M , O, O, O
```
The goal is to train an NLP model that predicts those labels correctly. In this project, BILUO labeling scheme is used.

# Goal

The goal of this project
1. Verify dataset, make sure that encoded batches are correct (misuse spans are correct). You can sample dataset and make sure that the number of errors is less than a certain threshold.
2. Train variable misuse detection model (with fine-tuning and without)
3. Verify [scoring function](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/entity/type_prediction.py#L71)
4. Conduct a series of experiments to identify performance
5. Analyze errors

# Why using this example?

Basic functionality, necessary for train an NLP labeler is
1. Loading data (implemented in this example)
2. Tokenization, preparing labels (implemented in [`PythonBatcher.prepare_sent`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L123))
3. Data encoding for using with ML models (implemented in [`PythonBatcher.create_batches_with_mask`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L206))
4. Batching (implemented in [`PythonBatcher.format_batch`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L256))
5. Model training (partially implemented in [`CodeBertModelTrainer2.train_model`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/codebert/codebert_train.py#L148) and extended here)
6. Tensorboard tracking (implemented in `CodeBertModelTrainer2`)

# Install libraries

1. See [installation steps](https://github.com/VitalyRomanov/method-embedding#installing-python-libraries).

2. Install transformers
```bash
pip install transformers
```

In [38]:
import os
import json
import torch
import logging
import pickle

from argparse import Namespace
from copy import copy
from datetime import datetime
from pathlib import Path
from os.path import join
from collections import defaultdict

import numpy as np
from transformers import RobertaModel, RobertaConfig
from tqdm import tqdm

from SourceCodeTools.nlp.codebert.codebert_train import CodeBertModelTrainer, get_length_mask
from SourceCodeTools.nlp.batchers.PythonBatcher import Batcher, PythonBatcher
from SourceCodeTools.code.data.cubert_python_benchmarks.data_iterators import DataIterator
from SourceCodeTools.nlp.entity.entity_scores import entity_scorer


# Definitions

## Reading Data

In [3]:
def read_data_fn_clf(dataset_path, partition):
    """
    Read data stored as JSON records.
    """
    assert partition in {"train", "val", "test"}
    data_path = join(dataset_path, f"var_misuse_seq_{partition}.json")

    for line in open(data_path, "r"):
        entry = json.loads(line)

        text = entry.pop("text")

        entry["category"] = "misuse" if len(entry["entities"]) > 0 else "correct"

        yield (text, entry)


class DataIteratorFnClf(DataIterator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __iter__(self):
        return read_data_fn_clf(self._data_path, self._partition_name)

## Model

In [765]:
from SourceCodeTools.mltools.torch import to_numpy
import torch.nn as nn

class CodebertHybridModelFnClf(nn.Module):
    def __init__(
            self, codebert_model, graph_emb, graph_padding_idx, num_classes, dense_hidden=100, dropout=0.1,
            bert_emb_size=768, no_graph=False
    ):
        super(CodebertHybridModelFnClf, self).__init__()

        self.codebert_model = codebert_model
        self.use_graph = not no_graph

        if self.use_graph:
            num_emb = graph_padding_idx + 1  # padding id is usually not a real embedding
            graph_emb_dim = graph_emb.shape[1]
            self.graph_emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=graph_emb_dim, padding_idx=graph_padding_idx)

            pretrained_embeddings = torch.from_numpy(np.concatenate([graph_emb, np.zeros((1, graph_emb_dim))], axis=0)).float()
            new_param = torch.nn.Parameter(pretrained_embeddings)
            assert self.graph_emb.weight.shape == new_param.shape
            self.graph_emb.weight = new_param
            self.graph_emb.weight.requires_grad = False
        else:
            graph_emb_dim = 0

        self.fc1 = nn.Linear(
            bert_emb_size + (graph_emb_dim if self.use_graph else 0),
            dense_hidden
        )
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(dense_hidden, num_classes)

        self.loss_f = nn.CrossEntropyLoss(reduction="mean")

    def forward(self, token_ids, graph_ids, mask, finetune=False):
        if finetune:
            x = self.codebert_model(input_ids=token_ids, attention_mask=mask).pooler_output
        else:
            with torch.no_grad():
                x = self.codebert_model(input_ids=token_ids, attention_mask=mask).pooler_output

        x = torch.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)

        return x

    def loss(self, logits, labels, mask, class_weights=None, extra_mask=None):
        loss = self.loss_f(logits, labels)
        return loss

    def score(self, logits, labels, mask, scorer=None, extra_mask=None):
        true_labels = labels
        estimated_labels = logits.argmax(-1)
        
        acc = (estimated_labels == true_labels).sum() / len(true_labels)

        return {"Accuracy": acc.cpu().item(), "Prediction": estimated_labels}

    def get_tensors_for_saliency(self, token_ids, graph_ids, mask):
        token_embs = self.codebert_model.embeddings.word_embeddings(token_ids)
        x = self.codebert_model(inputs_embeds=token_embs, attention_mask=mask).pooler_output
        x = torch.relu(self.fc1(x))
        x = self.drop(x)
        logits = self.fc2(x)
        
        return token_embs, logits

## Training procedure

In [869]:
class VariableMisuseDetector(CodeBertModelTrainer):

    def set_batcher_class(self):
        self.batcher = PythonBatcher

    def set_model_class(self):
        self.model = CodebertHybridModelFnClf

    @property
    def classes_for(self):
        return "labels"

    @property
    def best_score_metric(self):
        return "Accuracy"

    @classmethod
    def _format_batch(cls, batch, device):
        key_types = {
            'tok_ids': torch.LongTensor,
            'tags': torch.LongTensor,
            'hide_mask': torch.BoolTensor,
            'no_loc_mask': torch.BoolTensor,
            'lens': torch.LongTensor,
            'graph_ids': torch.LongTensor,
            'label': torch.LongTensor
        }
        for key, tf in key_types.items():
            if key in batch:
                batch[key] = tf(batch[key]).to(device)

    def get_training_dir(self):
        if not hasattr(self, "_timestamp"):
            self._timestamp = str(datetime.now()).replace(":", "-").replace(" ", "_")
        return Path(self.trainer_params["model_output"]).joinpath("codebert_var_mususe_fn_clf" + self._timestamp)
    
    def get_model(self, *args, **kwargs):
        codebert_model = RobertaModel.from_pretrained("microsoft/codebert-base")
        model = self.model(
            codebert_model, graph_emb=kwargs["graph_embedder"],
            graph_padding_idx=kwargs["graph_padding_idx"],
            num_classes=kwargs["num_classes"],
            no_graph=self.no_graph
        )
        if self.use_cuda:
            model.cuda()

        if self.ckpt_path is not None:
            ckpt_path = os.path.join(self.ckpt_path, "checkpoint")
            model = self.load_checkpoint(model, ckpt_path)
        return model

    def iterate_batches(self, model, batches, epoch, num_train_batches, train_scores, scorer, train=True):
        scores_for_averaging = defaultdict(list)

        batch_count = 0

        for ind, batch in enumerate(tqdm(batches, desc=f"Epoch {epoch}")):
            self._format_batch(batch, self.device)
            # Can get original tokens by calling
            # batches.get_record_with_id(batch["id"][0])
            scores = self.make_step(
                model=model, optimizer=self.optimizer, token_ids=batch['tok_ids'],
                prefix=batch['prefix'], suffix=batch['suffix'],
                graph_ids=batch['graph_ids'] if 'graph_ids' in batch else None,
                labels=batch['label'], lengths=batch['lens'],
                extra_mask=batch['no_loc_mask'] if self.no_localization else batch['hide_mask'],
                # class_weights=batch['class_weights'],
                scorer=scorer, finetune=self.finetune and epoch / self.epochs > 0.6,
                vocab_mapping=self.vocab_mapping,
                train=train
            )

            batch_count += 1

            scores["batch_size"] = batch['tok_ids'].shape[0]
            for score, value in scores.items():
                if "summary_writer" in self.__dict__:
                    self._write_to_summary(f"{score}/{'Train' if train else 'Test'}", value, epoch * num_train_batches + ind)
                scores_for_averaging[score].append(value)
            train_scores.append(scores_for_averaging)

        return num_train_batches
    
    def train_model(self):
        graph_emb = self._load_grap_embs()
        word_emb = self._load_word_embs()

        train_batcher, test_batcher = self.get_dataloaders(
            word_emb, graph_emb, self.suffix_prefix_buckets, cache_dir=Path(self.data_path).joinpath("__cache__")
        )

        trial_dir = self.get_training_dir()
        trial_dir.mkdir(parents=True, exist_ok=True)
        logging.info(f"Running trial: {str(trial_dir)}")
        self._create_summary_writer(trial_dir)

        self.save_params(
            trial_dir, {
                "MODEL_PARAMS": self.model_params,
                "TRAINER_PARAMS": self.trainer_params,
                "model_class": self.model.__class__.__name__,
                "batcher_class": self.batcher.__class__.__name__
            }
        )

        model = self.get_model(
            tok_embedder=word_emb, graph_embedder=graph_emb, train_embeddings=self.finetune,
            suffix_prefix_buckets=self.suffix_prefix_buckets,
            num_classes=train_batcher.num_classes(how=self.classes_for), seq_len=self.seq_len, no_graph=self.no_graph,
            graph_padding_idx=None,
            **self.model_params
        )

        def save_ckpt_fn():
            checkpoint_path = os.path.join(trial_dir, "checkpoint")
            self.save_checkpoint(model, checkpoint_path)

        train_scores, test_scores, train_average_scores, test_average_scores = self.train(
            model=model, train_batches=train_batcher, test_batches=test_batcher, epochs=self.epochs,
            learning_rate=self.learning_rate,
            scorer=lambda pred, true: entity_scorer(pred, true, train_batcher.tagmap,
                                                    no_localization=self.no_localization),
            learning_rate_decay=self.learning_rate_decay, finetune=self.finetune, save_ckpt_fn=save_ckpt_fn,
            no_localization=self.no_localization
        )

        metadata = {
            "train_scores": train_scores,
            "test_scores": test_scores,
            "train_average_scores": train_average_scores,
            "test_average_scores": test_average_scores,
        }

        with open(os.path.join(trial_dir, "train_data.json"), "w") as metadata_sink:
            metadata_sink.write(json.dumps(metadata, indent=4))

        pickle.dump(train_batcher.tagmap, open(os.path.join(trial_dir, "tag_types.pkl"), "wb"))

    def load_model(self, chkp_path):
        graph_emb = self._load_grap_embs()
        word_emb = self._load_word_embs()

        model = self.get_model(
            tok_embedder=word_emb, graph_embedder=graph_emb, train_embeddings=self.finetune,
            suffix_prefix_buckets=self.suffix_prefix_buckets,
            num_classes=2, seq_len=self.seq_len, no_graph=self.no_graph,
            graph_padding_idx=None,
            **self.model_params
        )
        model = torch.load(os.path.join(chkp_path, "checkpoint"))
        model = model.to(self.device)
        return model, graph_emb, word_emb

    def infer(self, model, graph_emb, word_emb):
        train_batcher, test_batcher = self.get_dataloaders(
            word_emb, graph_emb, self.suffix_prefix_buckets, cache_dir=Path(self.data_path).joinpath("__cache__")
        )

        scorer=lambda pred, true: entity_scorer(pred, true, test_batcher.tagmap,
                                                    no_localization=self.no_localization)

        batches = test_batcher
        output = []
        for ind, batch in enumerate(batches):
            self._format_batch(batch, self.device)
            tmp = [batches.get_record_with_id(int(i)) for i in batch["id"]]
            
            scores = self.make_step(
                model=model, optimizer=self.optimizer, token_ids=batch['tok_ids'],
                prefix=batch['prefix'], suffix=batch['suffix'],
                graph_ids=batch['graph_ids'] if 'graph_ids' in batch else None,
                labels=batch['label'], lengths=batch['lens'],
                extra_mask=batch['no_loc_mask'] if self.no_localization else batch['hide_mask'],
                scorer=scorer, finetune=False,
                vocab_mapping=self.vocab_mapping,
                train=False,
            )
            tmp = [(t, int(s)) for t, s in zip(tmp, scores["Prediction"])]
            output.extend(tmp)

        return output

    def sailency_map(self, model, graph_emb, word_emb):
        train_batcher, test_batcher = self.get_dataloaders(
            word_emb, graph_emb, self.suffix_prefix_buckets, cache_dir=Path(self.data_path).joinpath("__cache__")
        )

        batches = test_batcher
        records, output = [], []
        for ind, batch in enumerate(batches):
            self._format_batch(batch, self.device)
            record = [batches.get_record_with_id(int(i)) for i in batch["id"]]
            batch['tok_ids'][batch['tok_ids'] == 50265] = 1

            mask = get_length_mask(batch['tok_ids'], batch['lens'])
            token_embs, logits = model.get_tensors_for_saliency(
                batch['tok_ids'], batch['graph_ids'] if 'graph_ids' in batch else None, mask
            )
            loss = model.loss(logits, batch['label'], mask)

            token_embs.retain_grad()
            loss.backward()
            saliency = torch.mean(token_embs.grad.data.abs(), dim=2)

            records.extend(record)
            output.extend(saliency)

        return list(zip(records, output))

# Execution

All training options are specified [here](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/entity/type_prediction.py#L256)
Option names are added to `args` below.

In [870]:
dataset_path = "data/cubert_2_percent/"

args = Namespace()
args.__dict__.update({
    "learning_rate": 1e-6,           #
    "learning_rate_decay": 0.99,     #
    "max_seq_len": 512,              # default for BERT
    "random_seed": 42,               #
    "epochs": 2,                     #
    "gpu": -1,                       # set this to GPU id to use gpu
    "batch_size": 8,                 # higher value increases memory consumption
    "finetune": True,  # set this flag to enable finetuning
    "no_localization": False,        # whether to solve variable misuse with, or without localization
    
    # do not change items below
    "data_path": dataset_path,
    "no_graph": True,                # used for another model
    "model_output": dataset_path,    # where to store checkpoints
    "graph_emb_path": None,          # used for another model
    "word_emb_path": None,           # used for another model
    "trials": 1,                     # setting > 1 repeats training, used to accumulate statisitcs
    "suffix_prefix_buckets": 1,
    "mask_unlabeled_declarations": False,
    "ckpt_path": None
})

In [871]:
# train_data = DataIteratorFnClf(dataset_path, "train")
# test_data = DataIteratorFnClf(dataset_path, "val")

In [872]:
# test_data[0]  # ignore `replacements`

In [873]:
# trainer = VariableMisuseDetector(
#    train_data, test_data, model_params={}, trainer_params=copy(args.__dict__)
# )

In [874]:
#trainer.train_model()

In [875]:
# test_data = DataIteratorFnClf(dataset_path, "val")
# trainer.apply_model(test_data, "/Users/LTV/dev/method-embeddings/examples/variable_misuse_graph_2_percent_balanced/with_ast/codebert_var_mususe_fn_clf2022-11-01_14-46-38.717526")

# Inference

In [876]:
test_data = DataIteratorFnClf(dataset_path, "val")
test_data_10 = []
for i, d in enumerate(test_data):
    if i <= 1:
        continue
    elif i == 10:
        break
    test_data_10.append(d)

In [877]:
for t in test_data_10:
    print(t[0])
    print()

def save(self, request, contact_type=None):
    'Process form and create DB objects as required'
    if self.instance:
        contact = self.instance
    else:
        contact = Contact()
        contact.contact_type = contact_type
    contact.name = unicode(self.cleaned_data['name'])
    if ('parent' in self.cleaned_data):
        contact.parent = self.cleaned_data['parent']
    if ('related_user' in self.cleaned_data):
        contact.related_user = self.cleaned_data['related_user']
    contact.save()
    if self.instance:
        contact.contactvalue_set.all().delete()
    for field in contact.contact_type.fields.all():
        for form_name in self.cleaned_data:
            if re.match(str((('^' + field.name) + '___\\d+$')), form_name):
                if isinstance(self.fields[form_name], forms.FileField):
                    value = ContactValue(field=field, contact=contact, value=self._handle_uploaded_file(form_name))
                    if isinstance(self.fields[form_name], fo

In [878]:
trainer = VariableMisuseDetector([], test_data_10, model_params={}, trainer_params=copy(args.__dict__))

In [879]:
trainer.optimizer = None
model_args = trainer.load_model("codebert_var_mususe_fn_clf2022-10-27_17-05-40.996873")

In [888]:
# infer_pred = trainer.infer(*model_args)
for record, label in infer_pred:
    print("Code:")
    code = record["text"]

    if record["category"] == "misuse":
        i, j = record["labels"][0][:2]
        result = f"<span style='white-space: pre-wrap'>" + code[:i] + "</span>"
        result += f"<span style='white-space: pre-wrap;background-color:#FF000077'>{code[i:j]}</span>"
        result += f"<span style='white-space: pre-wrap'>" + code[j:] + "</span>"
        display(Markdown(result))
    else:
        print(code)

    label = "misuse" if label == 1 else "correct"
    print(f"Predicted: {label}, True label: {record['category']}")
    print()

Code:
def testExactNums(self):
    input = 'Remind me on January 26'
    target = datetime.datetime(2014, 1, 26)
    self.compareDate(input, target)
Predicted: correct, True label: correct

Code:


<span style='white-space: pre-wrap'>@classmethod
def select(cls, *selection):
    query = SelectQuery(cls, *selection)
    if cls._meta.order_by:
        query = query.order_by(*cls._meta.order_by)
    return </span><span style='white-space: pre-wrap;background-color:#FF000077'>cls</span><span style='white-space: pre-wrap'></span>

Predicted: misuse, True label: misuse

Code:


<span style='white-space: pre-wrap'>def add(self, obj, alias=None):
    if (obj in </span><span style='white-space: pre-wrap;background-color:#FF000077'>alias</span><span style='white-space: pre-wrap'>._alias_map):
        return
    self._counter += 1
    self._alias_map[obj] = (alias or ('%s%s' % (self.prefix, self._counter)))</span>

Predicted: misuse, True label: misuse

Code:
def execute(self, i, o):
    '\n        Executes the command.\n\n        :type i: cleo.inputs.input.Input\n        :type o: cleo.outputs.output.Output\n        '
    return self.handle()
Predicted: correct, True label: correct

Code:
def test_can_list_user_roles(self):
    with user_and_project_generator(self.manager) as (user, project):
        self.manager.add_role(user, 'sysadmin')
        roles = self.manager.get_user_roles(user)
        self.assertTrue(('sysadmin' in roles))
        self.assertFalse(('netadmin' in roles))
Predicted: correct, True label: correct

Code:
def _searchsorted_monotonic(self, label, side='left'):
    if self.is_monotonic_increasing:
        return self.searchsorted(label, side=side)
    elif self.is_monotonic_decreasing:
        pos = self[::(- 1)].searchsorted(label, side=('right' if (side == 'left') else 'right'))
        return (len(self) - pos)
    raise ValueError('index must be monotonic increasing or dec

In [823]:
sailency_map_pred = trainer.sailency_map(*model_args)

Scanning data: 0it [00:00, ?it/s]
Scanning data: 100%|███████████████████████████| 8/8 [00:00<00:00, 11785.89it/s]


In [892]:
from IPython.display import Markdown

def get_tokens(record):
    d = record["tokens"]._replacements
    res = []
    for t in record["tokens"].tokens:
        for k, v in d.items():
            t = t.replace(k, v)
        res.append(t)
    return res

def print_colorize(sample):
    tokens = get_tokens(sample[0])
    saliency_map = sample[1].tolist()
    s = np.std(saliency_map)

    result = ""
    for t, v in zip(tokens, saliency_map):
        a = int(11 * np.min([9, np.rint(np.round(1 + v / s))]))
        result += f"<span style='white-space: pre-wrap;background-color:#FF0000{a}'>{t}</span>"

    display(Markdown(result))
    print()

In [893]:
for x in sailency_map_pred:
    print_colorize(x)

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000022'>def</span><span style='white-space: pre-wrap;background-color:#FF000055'> test</span><span style='white-space: pre-wrap;background-color:#FF000022'>Ex</span><span style='white-space: pre-wrap;background-color:#FF000033'>act</span><span style='white-space: pre-wrap;background-color:#FF000022'>N</span><span style='white-space: pre-wrap;background-color:#FF000033'>ums</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000033'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000044'> input</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>Rem</span><span style='white-space: pre-wrap;background-color:#FF000022'>ind</span><span style='white-space: pre-wrap;background-color:#FF000022'> me</span><span style='white-space: pre-wrap;background-color:#FF000022'> on</span><span style='white-space: pre-wrap;background-color:#FF000033'> January</span><span style='white-space: pre-wrap;background-color:#FF000022'> 26</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000055'> target</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'> dat</span><span style='white-space: pre-wrap;background-color:#FF000022'>etime</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>dat</span><span style='white-space: pre-wrap;background-color:#FF000033'>etime</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000033'>2014</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> 1</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> 26</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000033'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'> </span><span style='white-space: pre-wrap;background-color:#FF000044'> self</span><span style='white-space: pre-wrap;background-color:#FF000033'>.</span><span style='white-space: pre-wrap;background-color:#FF000066'>comp</span><span style='white-space: pre-wrap;background-color:#FF000055'>are</span><span style='white-space: pre-wrap;background-color:#FF000055'>Date</span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000055'>input</span><span style='white-space: pre-wrap;background-color:#FF000033'>,</span><span style='white-space: pre-wrap;background-color:#FF000055'> target</span><span style='white-space: pre-wrap;background-color:#FF000033'>)</span><span style='white-space: pre-wrap;background-color:#FF000033'></span>

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000022'>@</span><span style='white-space: pre-wrap;background-color:#FF000022'>class</span><span style='white-space: pre-wrap;background-color:#FF000022'>method</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'>def</span><span style='white-space: pre-wrap;background-color:#FF000033'> select</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>cl</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>*</span><span style='white-space: pre-wrap;background-color:#FF000022'>selection</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000066'> query</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000033'> Select</span><span style='white-space: pre-wrap;background-color:#FF000044'>Query</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>cl</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>*</span><span style='white-space: pre-wrap;background-color:#FF000033'>selection</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> if</span><span style='white-space: pre-wrap;background-color:#FF000033'> cl</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>meta</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>order</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>by</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000066'> query</span><span style='white-space: pre-wrap;background-color:#FF000033'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>=</span><span style='white-space: pre-wrap;background-color:#FF000088'> query</span><span style='white-space: pre-wrap;background-color:#FF000033'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>order</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>by</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>*</span><span style='white-space: pre-wrap;background-color:#FF000022'>cl</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>meta</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>order</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>by</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'> return</span><span style='white-space: pre-wrap;background-color:#FF000055'> cl</span><span style='white-space: pre-wrap;background-color:#FF000055'>s</span><span style='white-space: pre-wrap;background-color:#FF000044'></span>

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000022'>def</span><span style='white-space: pre-wrap;background-color:#FF000033'> add</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000033'> obj</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000055'> alias</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000033'>None</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'> if</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000044'>obj</span><span style='white-space: pre-wrap;background-color:#FF000044'> in</span><span style='white-space: pre-wrap;background-color:#FF000099'> alias</span><span style='white-space: pre-wrap;background-color:#FF000055'>.</span><span style='white-space: pre-wrap;background-color:#FF000044'>_</span><span style='white-space: pre-wrap;background-color:#FF000044'>alias</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>map</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> return</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>counter</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>+</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'> 1</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000033'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>alias</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>map</span><span style='white-space: pre-wrap;background-color:#FF000033'>[</span><span style='white-space: pre-wrap;background-color:#FF000033'>obj</span><span style='white-space: pre-wrap;background-color:#FF000022'>]</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000055'>alias</span><span style='white-space: pre-wrap;background-color:#FF000033'> or</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000011'>%</span><span style='white-space: pre-wrap;background-color:#FF000011'>s</span><span style='white-space: pre-wrap;background-color:#FF000011'>%</span><span style='white-space: pre-wrap;background-color:#FF000011'>s</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>%</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>prefix</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>counter</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'></span>

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000033'>def</span><span style='white-space: pre-wrap;background-color:#FF000044'> execute</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000033'>self</span><span style='white-space: pre-wrap;background-color:#FF000033'>,</span><span style='white-space: pre-wrap;background-color:#FF000077'> i</span><span style='white-space: pre-wrap;background-color:#FF000044'>,</span><span style='white-space: pre-wrap;background-color:#FF000066'> o</span><span style='white-space: pre-wrap;background-color:#FF000033'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>\</span><span style='white-space: pre-wrap;background-color:#FF000011'>n</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> Exec</span><span style='white-space: pre-wrap;background-color:#FF000022'>utes</span><span style='white-space: pre-wrap;background-color:#FF000022'> the</span><span style='white-space: pre-wrap;background-color:#FF000033'> command</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>\</span><span style='white-space: pre-wrap;background-color:#FF000011'>n</span><span style='white-space: pre-wrap;background-color:#FF000022'>\</span><span style='white-space: pre-wrap;background-color:#FF000011'>n</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>:</span><span style='white-space: pre-wrap;background-color:#FF000033'>type</span><span style='white-space: pre-wrap;background-color:#FF000066'> i</span><span style='white-space: pre-wrap;background-color:#FF000033'>:</span><span style='white-space: pre-wrap;background-color:#FF000033'> cle</span><span style='white-space: pre-wrap;background-color:#FF000033'>o</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>input</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>input</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000044'>Input</span><span style='white-space: pre-wrap;background-color:#FF000022'>\</span><span style='white-space: pre-wrap;background-color:#FF000022'>n</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000033'>type</span><span style='white-space: pre-wrap;background-color:#FF000055'> o</span><span style='white-space: pre-wrap;background-color:#FF000033'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'> cle</span><span style='white-space: pre-wrap;background-color:#FF000033'>o</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>output</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>output</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000033'>Output</span><span style='white-space: pre-wrap;background-color:#FF000022'>\</span><span style='white-space: pre-wrap;background-color:#FF000022'>n</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> return</span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000055'>handle</span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'></span>

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000022'>def</span><span style='white-space: pre-wrap;background-color:#FF000022'> test</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>can</span><span style='white-space: pre-wrap;background-color:#FF000011'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>list</span><span style='white-space: pre-wrap;background-color:#FF000011'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>user</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>ro</span><span style='white-space: pre-wrap;background-color:#FF000022'>les</span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'> with</span><span style='white-space: pre-wrap;background-color:#FF000033'> user</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>and</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000055'>project</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000044'>gener</span><span style='white-space: pre-wrap;background-color:#FF000033'>ator</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>manager</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000044'> as</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000044'>user</span><span style='white-space: pre-wrap;background-color:#FF000044'>,</span><span style='white-space: pre-wrap;background-color:#FF000099'> project</span><span style='white-space: pre-wrap;background-color:#FF000033'>)</span><span style='white-space: pre-wrap;background-color:#FF000033'>:</span><span style='white-space: pre-wrap;background-color:#FF000033'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>manager</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>add</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>role</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000044'>user</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>sys</span><span style='white-space: pre-wrap;background-color:#FF000022'>admin</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> roles</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000011'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>manager</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>get</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>user</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>ro</span><span style='white-space: pre-wrap;background-color:#FF000022'>les</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000044'>user</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>assert</span><span style='white-space: pre-wrap;background-color:#FF000022'>True</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000033'>sys</span><span style='white-space: pre-wrap;background-color:#FF000022'>admin</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'> in</span><span style='white-space: pre-wrap;background-color:#FF000022'> roles</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000011'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>assert</span><span style='white-space: pre-wrap;background-color:#FF000022'>False</span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000033'>net</span><span style='white-space: pre-wrap;background-color:#FF000022'>admin</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'> in</span><span style='white-space: pre-wrap;background-color:#FF000033'> roles</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'></span>

<span style='white-space: pre-wrap;background-color:#FF000033'></span><span style='white-space: pre-wrap;background-color:#FF000022'>def</span><span style='white-space: pre-wrap;background-color:#FF000022'> _</span><span style='white-space: pre-wrap;background-color:#FF000033'>search</span><span style='white-space: pre-wrap;background-color:#FF000022'>s</span><span style='white-space: pre-wrap;background-color:#FF000033'>orted</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000011'>mon</span><span style='white-space: pre-wrap;background-color:#FF000011'>ot</span><span style='white-space: pre-wrap;background-color:#FF000022'>onic</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>self</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000044'> label</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> side</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>left</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000011'>)</span><span style='white-space: pre-wrap;background-color:#FF000011'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> if</span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000022'>is</span><span style='white-space: pre-wrap;background-color:#FF000011'>_</span><span style='white-space: pre-wrap;background-color:#FF000011'>mon</span><span style='white-space: pre-wrap;background-color:#FF000011'>ot</span><span style='white-space: pre-wrap;background-color:#FF000022'>onic</span><span style='white-space: pre-wrap;background-color:#FF000022'>_</span><span style='white-space: pre-wrap;background-color:#FF000033'>increasing</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> return</span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000044'>search</span><span style='white-space: pre-wrap;background-color:#FF000033'>s</span><span style='white-space: pre-wrap;background-color:#FF000044'>orted</span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000055'>label</span><span style='white-space: pre-wrap;background-color:#FF000022'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> side</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'>side</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> el</span><span style='white-space: pre-wrap;background-color:#FF000022'>if</span><span style='white-space: pre-wrap;background-color:#FF000022'> self</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000011'>is</span><span style='white-space: pre-wrap;background-color:#FF000011'>_</span><span style='white-space: pre-wrap;background-color:#FF000011'>mon</span><span style='white-space: pre-wrap;background-color:#FF000011'>ot</span><span style='white-space: pre-wrap;background-color:#FF000022'>onic</span><span style='white-space: pre-wrap;background-color:#FF000011'>_</span><span style='white-space: pre-wrap;background-color:#FF000022'>dec</span><span style='white-space: pre-wrap;background-color:#FF000022'>re</span><span style='white-space: pre-wrap;background-color:#FF000022'>asing</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000088'> pos</span><span style='white-space: pre-wrap;background-color:#FF000033'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>=</span><span style='white-space: pre-wrap;background-color:#FF000033'> self</span><span style='white-space: pre-wrap;background-color:#FF000033'>[</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>:</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>-</span><span style='white-space: pre-wrap;background-color:#FF000022'> 1</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>]</span><span style='white-space: pre-wrap;background-color:#FF000022'>.</span><span style='white-space: pre-wrap;background-color:#FF000044'>search</span><span style='white-space: pre-wrap;background-color:#FF000033'>s</span><span style='white-space: pre-wrap;background-color:#FF000044'>orted</span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000066'>label</span><span style='white-space: pre-wrap;background-color:#FF000033'>,</span><span style='white-space: pre-wrap;background-color:#FF000022'> side</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>right</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'> if</span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>side</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000022'>=</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>left</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'> else</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>right</span><span style='white-space: pre-wrap;background-color:#FF000011'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> </span><span style='white-space: pre-wrap;background-color:#FF000033'> return</span><span style='white-space: pre-wrap;background-color:#FF000044'> </span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000044'>len</span><span style='white-space: pre-wrap;background-color:#FF000033'>(</span><span style='white-space: pre-wrap;background-color:#FF000033'>self</span><span style='white-space: pre-wrap;background-color:#FF000033'>)</span><span style='white-space: pre-wrap;background-color:#FF000044'> </span><span style='white-space: pre-wrap;background-color:#FF000044'>-</span><span style='white-space: pre-wrap;background-color:#FF000088'> pos</span><span style='white-space: pre-wrap;background-color:#FF000033'>)</span><span style='white-space: pre-wrap;background-color:#FF000033'>
</span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000011'> </span><span style='white-space: pre-wrap;background-color:#FF000022'> raise</span><span style='white-space: pre-wrap;background-color:#FF000011'> Value</span><span style='white-space: pre-wrap;background-color:#FF000011'>Error</span><span style='white-space: pre-wrap;background-color:#FF000011'>(</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000033'>index</span><span style='white-space: pre-wrap;background-color:#FF000022'> must</span><span style='white-space: pre-wrap;background-color:#FF000011'> be</span><span style='white-space: pre-wrap;background-color:#FF000011'> mon</span><span style='white-space: pre-wrap;background-color:#FF000011'>ot</span><span style='white-space: pre-wrap;background-color:#FF000022'>onic</span><span style='white-space: pre-wrap;background-color:#FF000033'> increasing</span><span style='white-space: pre-wrap;background-color:#FF000022'> or</span><span style='white-space: pre-wrap;background-color:#FF000033'> decreasing</span><span style='white-space: pre-wrap;background-color:#FF000022'>'</span><span style='white-space: pre-wrap;background-color:#FF000022'>)</span><span style='white-space: pre-wrap;background-color:#FF000022'></span>